In [28]:
# загружаем данные
from os import path, environ
import pandas as pd
import numpy as np

data_dir = "/Users/kurzyakov/Desktop/Dr/2/ml-latest-small"
environ["DATA_DIR"] = "/Users/kurzyakov/Desktop/Dr/2/ml-latest-small"

col_names = ["user", "artist-mbid", "artist-name", "total-plays"]
data = pd.read_csv(
    path.join(data_dir, "lastfm_small.tsv"),
    sep="\t",
    header=None,
    names=["user", "artist-mbid", "artist-name", "total-plays"]
)
data.head()

,user,artist-mbid,artist-name,total-plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [29]:
# заполняем пустые значения
data.fillna("None", inplace=True)
# заменим строковые идентификаторы числовыми кодами
# добавляем к индексам единицы, потому что в mrec,
# который будем использовать для оценки качества, индексы начинаются с единицы
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
# убираем лишние колонки
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

,total-plays,user_id,artist_id
0,2137,1,15531
1,1099,1,63469
2,897,1,46858
3,717,1,15968
4,706,1,48969


In [19]:
#!pip install implicit

  Using cached implicit-0.2.7.tar.gz
  Running setup.py bdist_wheel for implicit ... done
  Stored in directory: /Users/kurzyakov/Library/Caches/pip/wheels/9a/6e/e6/71106a349ec65d9024a2832d8ca94a2225c8c271b44bb742c8
Successfully built implicit


In [30]:
data.describe()

,total-plays,user_id,artist_id
count,1000000.000000,1000000.000000,1000000.000000
mean,216.606950,10232.925996,33678.492236
std,604.378024,5912.022447,19230.330182
min,1.000000,1.000000,1.000000
25%,34.000000,5118.000000,17298.000000
50%,94.000000,10237.000000,34544.000000
75%,225.000000,15347.000000,49488.000000
max,135392.000000,20465.000000,66799.000000


In [112]:
# разобьём наблюдения на тестовую и обучающую выборки
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.4)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [113]:
test_user_set = set(test_data["user_id"].unique()) #set выдает элементы заданного множества, только 1 раз
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 6
нет в тестовой выборке, но есть в обучающей: 7
всего пользователей: 20465


In [114]:
# исключим таких пользователей из тестовой и обучающей выборок
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index
test_data.drop(bad_indices, inplace=True)
bad_indices = train_data[train_data["user_id"].isin(user_ids_to_exclude).values]
train_data.drop(bad_indices.index, inplace=True)

/Users/kurzyakov/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [115]:
test_file_name = "lastfm.test.0"
test_data[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, test_file_name),
    sep="\t",
    header=False,
    index=False
)
train_file_name = "lastfm.train.0"
train_data[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, train_file_name),
    sep="\t",
    header=False,
    index=False
)

In [106]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

SyntaxError: invalid syntax (<ipython-input-106-fb7245614083>, line 4)

In [116]:
# создаём разреженную матрицу item*user
from scipy.sparse import coo_matrix
import numpy as np

#для кос. меры
plays = coo_matrix((
    train_data["total-plays"].astype(np.float32),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

#для als 
plays1 = coo_matrix((
    train_data["total-plays"].astype('double'),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

#sparse_info(plays.tocsr())

In [117]:
import time
from implicit.nearest_neighbours import CosineRecommender

model = CosineRecommender(K=30)
print("строим матрицу схожести по косинусной мере")
start = time.time()
model.fit(plays)
print("построили матрицу схожести по косинусной мере за {} секунд".format(
        time.time() - start))
#sparse_info(model.similarity)

строим матрицу схожести по косинусной мере
построили матрицу схожести по косинусной мере за 1.16793107986 секунд


In [118]:
print("получаем рекомендации для всех пользователей по косинусной мере")
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(data_dir, "recs", test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))



получаем рекомендации для всех пользователей по косинусной мере
получили рекомендации для всех пользователей за 21.3301320076 секнуд


In [123]:
from implicit import als 
model1 = als.AlternatingLeastSquares(factors = 150, iterations = 100)
print("строим матрицу схожести с помощью als")
model1.fit(plays1)

строим матрицу схожести с помощью als


In [124]:
print("получаем рекомендации для всех пользователей с помощью als")
start = time.time()
user_plays1 = plays1.T.tocsr()
with open(path.join(data_dir, "recs1", test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model1.recommend(user_id, user_plays1):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))


получаем рекомендации для всех пользователей с помощью als
получили рекомендации для всех пользователей за 121.056670904 секнуд


In [119]:
# используем mrec в качестве метрики качества для kNN
!mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/recs

[2017-12-05 18:06:48,664] INFO: processing /Users/kurzyakov/Desktop/Dr/2/ml-latest-small/lastfm.test.0...
None
mrr            0.0630 +/- 0.0000
prec@5         0.0226 +/- 0.0000
prec@10        0.0225 +/- 0.0000
prec@15        0.0150 +/- 0.0000
prec@20        0.0112 +/- 0.0000


In [125]:
# используем mrec в качестве метрики качества для als
!mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/recs1

[2017-12-05 18:21:10,447] INFO: processing /Users/kurzyakov/Desktop/Dr/2/ml-latest-small/lastfm.test.0...
None
mrr            0.5025 +/- 0.0000
prec@5         0.2618 +/- 0.0000
prec@10        0.2179 +/- 0.0000
prec@15        0.1453 +/- 0.0000
prec@20        0.1090 +/- 0.0000
